In [6]:
import h5py
import numpy as np
from sklearn.utils import shuffle


# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [9]:

X_train = []
X_test = []

for filename in ['D:\data\kaggle\cat_dog\my_gap\gap_my_ResNet50.h5', 
                #  'D:\data\kaggle\cat_dog\my_gap\gap_my_Xception.h5', 
                #  'D:\data\kaggle\cat_dog\my_gap\gap_my_InceptionV3.h5'
                 ]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)

In [10]:
from keras.models import *
from keras.layers import *

input_tensor = Input(X_train.shape[1:])
x = input_tensor
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [4]:
# 使用load_model函数加载模型

model = load_model(r"C:\Users\haokw\Documents\GitHub\VScode\cat_dog\model\model_my_best_3.h5")

# 查看模型的架构
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 6144)]            0         
                                                                 
 dropout (Dropout)           (None, 6144)              0         
                                                                 
 dense (Dense)               (None, 1)                 6145      
                                                                 
Total params: 6,145
Trainable params: 6,145
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(X_train, y_train, batch_size=512, epochs=100, validation_split=0.2)

Epoch 1/100
40/40 [==============================] - 1s 25ms/step - loss: 0.9683 - accuracy: 0.4746 - val_loss: 0.8122 - val_accuracy: 0.4634
Epoch 2/100
40/40 [==============================] - 1s 21ms/step - loss: 0.9436 - accuracy: 0.4837 - val_loss: 0.8055 - val_accuracy: 0.4690
Epoch 3/100
40/40 [==============================] - 1s 23ms/step - loss: 0.9606 - accuracy: 0.4755 - val_loss: 0.7988 - val_accuracy: 0.4756
Epoch 4/100
40/40 [==============================] - 1s 16ms/step - loss: 0.9425 - accuracy: 0.4836 - val_loss: 0.7923 - val_accuracy: 0.4802
Epoch 5/100
40/40 [==============================] - 1s 16ms/step - loss: 0.9354 - accuracy: 0.4922 - val_loss: 0.7856 - val_accuracy: 0.4864
Epoch 6/100
40/40 [==============================] - 1s 24ms/step - loss: 0.9262 - accuracy: 0.4904 - val_loss: 0.7790 - val_accuracy: 0.4940
Epoch 7/100
40/40 [==============================] - 1s 23ms/step - loss: 0.9301 - accuracy: 0.4946 - val_loss: 0.7724 - val_accuracy: 0.4998
Epoch 

In [ ]:
# model.save('model\\model_myyy_5.h5')

In [ ]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

In [ ]:
import pandas as pd
from keras.preprocessing.image import *

df = pd.read_csv("result\pred.csv")
df2 = pd.read_csv("result\pred_name.csv")
image_size = (224, 224)
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(r"D:\data\kaggle\cat_dog\test", image_size, shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('\\')+1:fname.rfind('.')])
    df.at[index-1, 'label'] = y_pred[i]
    if y_pred[i]>0.50:
        df2.at[index-1, 'label'] = "狗"
    else:
        df2.at[index-1, 'label'] = "猫"


df.to_csv('result\\pred.csv', index=None)
df.head(10)
df2.to_csv('result\\pred_name.csv', index=None)
df2.head(10)